# KNN

### What is KNN?

- Supervised Algorithm
- Makes predictions based on how close a new data point is to known data points.
- Lazy
- Sensitive to scaling

Link: [KNN Diagram](https://cambridgecoding.files.wordpress.com/2016/01/knn2.jpg)

#### Pros:
1. Simple
2. Robust to noise
3. Effective with large datasets
4. Performs calculations "just in time"
5. Data is easy to keep up to date to keep predictions accurate

#### Cons:
1. Need to determine how many neighbors is optimal
2. Computation cost is high (has to calculate every single distance to every feature)
3. Euclidean volume increases exponentially as number of features increases (curse of dimensionality)

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Data handling
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

# Data acquisition
from pydataset import data

# Data modeling
from model import map_setosa_knn, train_validate_test_split

## Acquire (Iris Dataset)

In [ ]:
# read Iris data from pydatset
df = data('iris')

# convert column names to lowercase, replace '.' in column names with '_'
df.columns = [col.lower().replace('.', '_') for col in df]

df.head()

In [ ]:
df.info()

In [ ]:
df.species.value_counts()

## Prepare/Preprocessing

In [ ]:
# split into train, validate, test
train, validate, test = train_validate_test_split(df, target='species', seed=123)

# create X & y version of train/validate/test
# where X contains the features we want to use and y is a series with just the target variable

X_train = train.drop(columns=['species', 'petal_length', 'petal_width'])
y_train = train.species

X_validate = validate.drop(columns=['species', 'petal_length', 'petal_width'])
y_validate = validate.species

X_test = test.drop(columns=['species', 'petal_length', 'petal_width'])
y_test = test.species

In [ ]:
sns.scatterplot(data=X_train, x='sepal_length', y='sepal_width')

## Train Model

#### Create KNN Object

In [ ]:
knn1 = KNeighborsClassifier(n_neighbors=1, weights='uniform')

#### Fit the Model to the Training Data

In [ ]:
knn1.fit(X_train, y_train)

#### Make Predictions

In [ ]:
y_pred = knn1.predict(X_train)
y_pred[:5]

In [ ]:
y_train[:5]

#### Estimate Probability

In [ ]:
y_pred_proba = knn1.predict_proba(X_train)
y_pred_proba[:5]

## Evaluate Model

**Note**: This next visualization is from a custom module (model.py). It has significant limitations and should not be used outside of this tutorial. Unfortunately, there is not an easy way to visualize KNN models. Many KNN models will be impossible to visually represent on a 2-D or 3-D plane anyways, due to the number of features. 

In [ ]:
map_setosa_knn(X_train, y_train, knn1)

#### Compute the Accuracy

In [ ]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn1.score(X_train, y_train)))

#### Create a Confusion Matrix

In [ ]:
print(confusion_matrix(y_train, y_pred))

In [ ]:
pd.crosstab(y_train, y_pred)

In [ ]:
plot_confusion_matrix(knn1, X_train, y_train)

#### Create a Classification Report

In [ ]:
TN, FP, FN, TP = confusion_matrix(y_train, y_pred).ravel() # Why is this line of code failing?

In [ ]:
print(classification_report(y_train, y_pred))

## Lets Do it Again

In [ ]:
# Create KNN Object
knn5 = KNeighborsClassifier(n_neighbors=5, weights='uniform')

# Fit object to training data
knn5.fit(X_train, y_train)

# Make predictions on training data
y_pred5 = knn5.predict(X_train)

How does the boundary map for **k = 5** compared to **k = 1**?

In [ ]:
map_setosa_knn(X_train, y_train, knn5)

In [ ]:
map_setosa_knn(X_train, y_train, knn1)

In [ ]:
# Evaluate the new model

print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn5.score(X_train, y_train)))

In [ ]:
plot_confusion_matrix(knn5, X_train, y_train)

It seems like our `knn1` model is better than our `knn5` model. But lets see how they perform on out-of-sample data

In [ ]:
print('Accuracy of KNN (k=1) classifier on validate set: {:.2f}'
     .format(knn1.score(X_validate, y_validate)))

print('Accuracy of KNN (k=5) classifier on validate set: {:.2f}'
     .format(knn5.score(X_validate, y_validate)))

Our second model is more performant on out of sample data. Furthermore, it has less of a drop between train score and validate score.

## Finding the Best value for k

In [ ]:
k_range = range(1, 20)
train_scores = []
validate_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    train_scores.append(knn.score(X_train, y_train))
    validate_scores.append(knn.score(X_validate, y_validate))
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.plot(k_range, train_scores, label='Train')
plt.plot(k_range, validate_scores, label='Validate')
plt.legend()
plt.xticks([0,5,10,15,20])
plt.show()

## Next Steps
- We only used sepal_length and sepal_width. We can try new models with different and/or additional features. 


- There are other hyperparameters we can tweak
    - 'weights': Uniform is the default (all points are treated equally), but we can switch to a 'distance' approach where nearer neighbors are given more weight in the voting process
    - 'algorithm': Large datasets use a sampling algorithm to save on computational cost. We can try different samplers. 
    - 'metric': There is more than one way to measure distance


- There are very similar models that we can try (RadiusNeighborsClassifier)

# Exercises

Continue working in your `model.ipynb` file with the titanic dataset.

1. Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

2. Evaluate your results using the model score, confusion matrix, and classification report.

3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

4. Run through steps 2-4 setting k to 10

5. Run through setps 2-4 setting k to 20

6. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

7. Which model performs best on our out-of-sample data from validate?

-------

Once you have completed work on the titanic dataset, try building some knn models with your telco data.